In [93]:
import re
import random
import string

# Fuknce pro znáhodnění textu
def random_string(length=12):
    chars = string.ascii_letters + string.digits
    return ''.join(random.choices(chars, k=length))

# Načtení vstupního souboru
with open("název_dat.json", "r", encoding="utf-8") as f:
    content = f.read()

# Hledání obasahu v <csv> </csv>
csv_match = re.search(r"<csv>(.*?)</csv>", content, re.DOTALL)
if not csv_match:
    print("CSV tag nenalezen.")
    exit()
csv_text = csv_match.group(1)
csv_lines = csv_text.strip().splitlines()

# První řádek zachovat jako header
header = csv_lines[0]
entries = []
# Rozdělení po řádcích a zachování čárek a enterů na konco řádku
for line in csv_lines[1:]:
    entries.extend([e.strip() for e in line.split(',') if e.strip()])

# Generování náhodných náhrad
replacements = {entry: random_string() for entry in set(entries)}

# Nahrazování výskytů v celém souboru
def replace_words(text, mapping):
    sorted_keys = sorted(mapping.keys(), key=len, reverse=True) 
    for k in sorted_keys:
        text = re.sub(re.escape(k), mapping[k], text)
    return text

# Nahradit mimo <csv> nejdřív (nahrazení stejných hodnot v části response (výsledky apri
before_csv = content[:csv_match.start()]
after_csv = content[csv_match.end():]
before_csv = replace_words(before_csv, replacements)
after_csv = replace_words(after_csv, replacements)

# Sestavit novou CSV sekci
csv_body = '\n'.join([
    ','.join(replacements.get(e.strip(), e.strip()) for e in line.split(','))
    for line in csv_lines[1:]
])
csv_final = f"<csv>\n{header}\n{csv_body}\n</csv>"

# Sestav finální text
final_content = before_csv + csv_final + after_csv

# === Ulož výsledek ===
with open("vystup_final_zachovano_header.txt", "w", encoding="utf-8") as f:
    f.write(final_content)

print("Hotovo! Header zachován a náhrady provedeny.")


Hotovo! Header zachován a náhrady provedeny.
